# Supervised Machine Learning Pipeline - Multi-class Classification

# Overview

Use activity recognition test dataset to:
1. Perform multi-class classification of activity recognition tasks (6 classes) using 131 features in the time and frequency domain.
2. Compare linear classifiers using a machine learning pipeline.

Linear classifiers
- k nearest neighbors
- logistic regression
- SVM - linearSVC
- SVM - SVC which uses nonlinear SVM by default

# Import packages

In [211]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import sklearn.datasets
import pandas as pd
import numpy as np
import seaborn as sns

# Loading data

In [353]:
# load test set
testfile = r'//FS2.smpp.local\RTO\Inpatient Sensors -Stroke\Data analysis\Analysis_ActivityRecognition\testfeaturematrix.csv'
df = pd.read_csv(testfile)

In [354]:
# once I have X and y, modify below
df.insert(8, 'target_category', df['task'].astype('category').cat.codes)
X = df.drop(df.columns[0:9], axis=1)
y = df['target_category']

# stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Should I scale before I split?

## How do I get rid of rank effect of numerical target labels?
- does rank matter for targets

In [355]:
df.head(1)

,subject,date,test,task,trial,location,sensor,rawdata,target_category,meanX,...,meanpower_bin11_z,meanpower_bin12_z,meanpower_bin13_z,meanpower_bin14_z,meanpower_bin15_z,meanpower_bin16_z,meanpower_bin17_z,meanpower_bin18_z,meanpower_bin19_z,meanpower_bin20_z
0,HC02,temp date,activity recognition,LYING,0,tibialis_anterior_left,accel,Accel X (g) Accel Y (g) Acce...,0,-0.049779,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000001,0.000002,0.000002,0.000001


In [356]:
print(X.shape, y.shape)

(242, 131) (242,)


In [357]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(193, 131) (193,) (49, 131) (49,)


Task code:
- Lying 0
- Sitting 1
- Stairs dn 2
- Stairs up 3
- Standing 4
- Walking 5

## One hot encoding - Dealing with categorical features
- scikit-learn: OneHotEncoder()
- pandas: get_dummies()

In [ ]:
OneHotEncoder()
LabelEncoder()

In [46]:
# create dummy variables and drop first column since it can be implied
y_dummy = pd.get_dummies(y, drop_first=True)
print(y_dummy.head(3))


   task_LYING  task_SITTING  task_STAIRS DOWN  task_STAIRS UP  task_STANDING
0           1             0                 0               0              0
1           1             0                 0               0              0
2           1             0                 0               0              0


In [233]:
# one-hot encoding

values = df['task'].values
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [228]:
from keras.utils import to_categorical
Y = df['task']
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)


In [229]:
dummy_y

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [232]:
np.array_equal(dummy_y,onehot_encoded)

True

In [281]:
X_train, X_test, y_train, y_test = train_test_split(X, onehot_encoded, random_state=42, stratify=y)

# Exploratory data analysis (EDA)

In [304]:
# check dimensions
print('AR data dimensions: ', X.shape)
print('AR target dimensions: ', y.shape)

AR data dimensions:  (242, 131)
AR target dimensions:  (242,)


In [305]:
# check dimensions
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(181, 131) (181,) (61, 131) (61,)


# Test using UCI HAR dataset

In [145]:
cd

C:\Users\lu-RT&O


In [146]:
ucixpath = r'C:\Users\lu-RT&O\Documents\GitHub\UCI HAR Dataset\train\X_train.txt'
uciypath = r'C:\Users\lu-RT&O\Documents\GitHub\UCI HAR Dataset\train\y_train.txt'
xdf = pd.read_fwf(ucixpath)
ydf = pd.read_fwf(uciypath)

In [156]:
# target
ydf['5'].unique()

array([5, 4, 6, 1, 3, 2], dtype=int64)

# 1. k nearest neighbors (knn)

X = df.drop(df.columns[0:9], axis=1)
y = df['target_category']

# stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [306]:
from sklearn.neighbors import KNeighborsClassifier

# Create and fit the model with default hyperparameters
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [307]:
y_train.shape

(181,)

In [308]:
y.shape

(242,)

In [309]:
# n=5
knn.score(X_test, y_test)

0.6229508196721312

In [310]:
# Predict on the test features, print the results
pred = knn.predict(X_test)[0]
# Prediction for test example 0: 1.0
print("Prediction for test example 0:", pred)

Prediction for test example 0: 4


In [311]:
# test k=6 with parameter n_neighbors
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.5245901639344263

In [319]:
# hyperparameter tuning for n_neighbors
# setting n to 1-30, overfits with n=1 ##############
param_grid = {'n_neighbors': np.arange(3, 30)}

# Option 1
knn = KNeighborsClassifier()
# Option 2
# steps = [('scaler', StandardScaler()),
#          ('knn', KNeighborsClassifier())]
# pipeline = Pipeline(steps)

# args: model, grid, number of folds for cross validation
knn_cv = GridSearchCV(knn, param_grid, cv=5)
knn_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned KNN Parameters: {}".format(knn_cv.best_params_)) 
print("Best score is {}".format(knn_cv.best_score_))

print("knn training accuracy:", knn_cv.score(X_train, y_train))
print("knn test accuracy    :", knn_cv.score(X_test, y_test))

Tuned KNN Parameters: {'n_neighbors': 9}
Best score is 0.580110497237569
knn training accuracy: 0.6850828729281768
knn test accuracy    : 0.5573770491803278


## knn Overfit
- add scaling

# 2. Logistic Regression - multi-class
Key hyperparameters:
- C (inverse regularization strength)
- penalty (type of regularization - L1 and L2)
- multi_class (type of multi-class)

## 2.1 One-vs-Rest

In [321]:
# LogisticRegression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
# lr.predict(X_test)
# lr.score(X_test, y_test)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [322]:
lr.score(X_test, y_test)

0.639344262295082

In [323]:
# Fit one-vs-rest logistic regression classifier
# lr_ovr = LogisticRegression()
lr_ovr = OneVsRestClassifier(LogisticRegression()) 
lr_ovr.fit(X_train, y_train)

print("OVR training accuracy:", lr_ovr.score(X_train, y_train))
print("OVR test accuracy    :", lr_ovr.score(X_test, y_test))

OVR training accuracy: 0.9171270718232044
OVR test accuracy    : 0.639344262295082


In [324]:
y_pred_lr_ovr = lr_ovr.predict(X_test)
print(classification_report(y_test, y_pred_lr_ovr))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_lr_ovr))

             precision    recall  f1-score   support

          0       0.50      0.33      0.40         6
          1       0.58      0.64      0.61        11
          2       0.33      0.60      0.43         5
          3       0.75      0.50      0.60         6
          4       0.73      0.73      0.73        22
          5       0.80      0.73      0.76        11

avg / total       0.66      0.64      0.64        61

Confusion matrix:
 [[ 2  2  0  0  2  0]
 [ 1  7  0  0  3  0]
 [ 0  0  3  0  1  1]
 [ 0  0  3  3  0  0]
 [ 1  3  1  0 16  1]
 [ 0  0  2  1  0  8]]


Task code:
- Lying 0
- Sitting 1
- Stairs dn 2
- Stairs up 3
- Standing 4
- Walking 5

Lying confused as Sitting 2
Lying confused with Standing 2
Sitting confused as Lying 1
Sitting confused with Standing 3
Stairs dn confused with Standing 1
Stairs dn confused with Walking 1
Standing confused with Lying 1
Standing confused with Sitting 3
Standing confused with Stairs dn 1
Standing confused with Walking 1
Walking confused as Stairs dn 2
Walking confused as Stairs up 1

### 2.1.a sample confidence intervals

In [ ]:
# confidence intervals
lr.predict_proba(X_test[:1])
# array([[ 9.951e-01, 4.357e-03, 5.339e-04]])

## 2.2 Softmax/Multinomial/Cross-Entropy Loss

In [325]:
lr_mn = LogisticRegression(multi_class="multinomial",solver="lbfgs")
lr_mn.fit(X_train, y_train)

print("Softmax training accuracy:", lr_mn.score(X_train, y_train))
print("Softmax test accuracy    :", lr_mn.score(X_test, y_test))

Softmax training accuracy: 0.7790055248618785
Softmax test accuracy    : 0.5737704918032787


In [326]:
y_pred_lr_mn = lr_mn.predict(X_test)
print(classification_report(y_test, y_pred_lr_mn))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_lr_mn))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         6
          1       0.29      0.18      0.22        11
          2       0.30      0.60      0.40         5
          3       0.50      0.17      0.25         6
          4       0.63      0.86      0.73        22
          5       0.83      0.91      0.87        11

avg / total       0.50      0.57      0.52        61

Confusion matrix:
 [[ 0  4  0  0  2  0]
 [ 0  2  0  0  9  0]
 [ 0  0  3  1  0  1]
 [ 0  0  5  1  0  0]
 [ 0  1  1  0 19  1]
 [ 0  0  1  0  0 10]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Optional: L1 regularization

In [328]:
# Specify L1 regularization
lr = LogisticRegression(penalty='l1')

# Instantiate the GridSearchCV object and run the search
searcher = GridSearchCV(lr, {'C':[0.001, 0.01, 0.1, 1, 10, 100]})
searcher.fit(X_train, y_train)

# Report the best parameters
print("Best CV params", searcher.best_params_)

# Find the number of nonzero coefficients (selected features)
best_lr = searcher.best_estimator_
coefs = best_lr.coef_
print("Total number of features:", coefs.size)
print("Number of selected features:", np.count_nonzero(coefs))

# with l1 reg - C=1
# without l1 reg - C=10

Best CV params {'C': 100}
Total number of features: 786
Number of selected features: 218


In [348]:
print("L1 reg training accuracy:", searcher.score(X_train, y_train))
print("L1 reg test accuracy    :", searcher.score(X_test, y_test))

L1 reg training accuracy: 0.7734806629834254
L1 reg test accuracy    : 0.5573770491803278


# 3. LinearSVC for SVM

In [330]:
# LinearSVC
import sklearn.datasets
wine = sklearn.datasets.load_wine()
from sklearn.svm import LinearSVC

# svm.ft(wine.data, wine.target)
# svm.score(wine.data, wine.target)

In [331]:
linearsvm = LinearSVC()
linearsvm.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [332]:
print("LinearSVC training accuracy:", linearsvm.score(X_train, y_train))
print("LinearSVC test accuracy    :", linearsvm.score(X_test, y_test))

LinearSVC training accuracy: 0.7679558011049724
LinearSVC test accuracy    : 0.6065573770491803


In [333]:
LinearSVC().get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'loss', 'max_iter', 'multi_class', 'penalty', 'random_state', 'tol', 'verbose'])

In [334]:
# CV and scaling in a pipeline using Normalization
steps = [('scaler', StandardScaler()),
         ('svc', LinearSVC())]
pipeline = Pipeline(steps)

# Specify hyperparameter space using a dictionary
parameters = {'svc__C':[0.1, 1, 10]}

X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train_svc, y_train_svc)
y_pred_svc = cv.predict(X_test_svc)

# Compute and print metrics
# print best parameters
print("Tuned Model Parameters: {}".format(cv.best_params_))
print("LinearSVC training accuracy:", linearsvm.score(X_train, y_train))
print("Test Accuracy: {}".format(cv.score(X_test_svc, y_test_svc)))
print(classification_report(y_test_svc, y_pred_svc))
print("Confusion matrix:\n", confusion_matrix(y_test_svc, y_pred_svc))

Tuned Model Parameters: {'svc__C': 0.1}
LinearSVC training accuracy: 0.7679558011049724
Test Accuracy: 0.7551020408163265
             precision    recall  f1-score   support

          0       0.80      1.00      0.89         4
          1       0.50      0.22      0.31         9
          2       0.60      0.75      0.67         4
          3       1.00      0.60      0.75         5
          4       0.73      0.89      0.80        18
          5       0.90      1.00      0.95         9

avg / total       0.74      0.76      0.73        49

Confusion matrix:
 [[ 4  0  0  0  0  0]
 [ 1  2  0  0  6  0]
 [ 0  0  3  0  0  1]
 [ 0  0  2  3  0  0]
 [ 0  2  0  0 16  0]
 [ 0  0  0  0  0  9]]


Sitting confused as Lying 1
Sitting confused with Standing 6
Stairs dn confused with Walking 1
Standing confused with Sitting 2

Task code:
Lying 0
Sitting 1
Stairs dn 2
Stairs up 3
Standing 4
Walking 5

# 4. SVC - default nonlinear SVM

In [335]:
# SVC
import sklearn.datasets
wine = sklearn.datasets.load_wine()
from sklearn.svm import SVC
svm = SVC()
# svm.fit(wine.data, wine.target)
# svm.score(wine.data, wine.target)

In [336]:
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [337]:
print("SVC training accuracy:", svm.score(X_train, y_train))
print("SVC test accuracy    :", svm.score(X_test, y_test))

# overfit model

SVC training accuracy: 0.861878453038674
SVC test accuracy    : 0.5737704918032787


## 4.1 SVC: Tune hyperparameters to improve test accuracy

In [338]:
# Instantiate an RBF SVM
svm = SVC()

# Instantiate the GridSearchCV object and run the search
parameters = {'C':[0.1, 1, 10], 'gamma':[0.00001, 0.0001, 0.001, 0.01, 0.1]}
searcher = GridSearchCV(svm, parameters)
searcher.fit(X_train,y_train)

# Report the best parameters and the corresponding score
print("Best CV params", searcher.best_params_)
print("Best CV accuracy", searcher.best_score_)

# Report the test accuracy using these best parameters
print("Train accuracy of best grid search hypers:", 
      searcher.score(X_train,y_train))
# Report the test accuracy using these best parameters
print("Test accuracy of best grid search hypers:", 
      searcher.score(X_test, y_test))

Best CV params {'C': 10, 'gamma': 1e-05}
Best CV accuracy 0.6132596685082873
Train accuracy of best grid search hypers: 0.7734806629834254
Test accuracy of best grid search hypers: 0.5573770491803278


## 4.2 SVC: normalize data, tune hyperparameters and check final result

In [352]:
# CV and scaling in a pipeline
steps = [('scaler', StandardScaler()),
         ('svm', SVC())]
pipeline = Pipeline(steps)

# Specify hyperparameter space using a dictionary
parameters = {'svm__C':[0.1, 1, 10],
              'svm__gamma':[0.00001, 0.0001, 0.001, 0.01, 0.1]}

X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train_svm, y_train_svm)
y_pred_svm = cv.predict(X_test_svm)

# Compute and print metrics
# print best parameters
print("Tuned Model Parameters: {}".format(cv.best_params_))
print("Test Accuracy: {}".format(cv.score(X_test_svm, y_test_svm)))
print(classification_report(y_test_svm, y_pred_svm))
print("Confusion matrix:\n", confusion_matrix(y_test_svm, y_pred_svm))

Tuned Model Parameters: {'svm__C': 10, 'svm__gamma': 0.01}
Test Accuracy: 0.6122448979591837
             precision    recall  f1-score   support

          0       0.75      0.75      0.75         4
          1       0.44      0.44      0.44         9
          2       0.00      0.00      0.00         4
          3       0.00      0.00      0.00         5
          4       0.78      0.78      0.78        18
          5       0.56      1.00      0.72         9

avg / total       0.53      0.61      0.56        49

Confusion matrix:
 [[ 3  1  0  0  0  0]
 [ 1  4  0  0  4  0]
 [ 0  0  0  0  0  4]
 [ 0  0  2  0  0  3]
 [ 0  4  0  0 14  0]
 [ 0  0  0  0  0  9]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Lying confused as Sitting 1
Lying confused with Standing 0
Sitting confused as Lying 1
Sitting confused with Standing 4
Stairs dn confused with Standing 0
Stairs dn confused with Walking 4
Stairs up confused with Stairs dn 2
Stairs up confused with Walking 3
Standing confused with Lying 0
Standing confused with Sitting 4
Standing confused with Stairs dn 0
Standing confused with Walking 0
Walking confused as Stairs dn 0
Walking confused as Stairs up 0

Task code:
Lying 0
Sitting 1
Stairs dn 2
Stairs up 3
Standing 4
Walking 5

# 5. SGDClassifier (SGD=stochastic gradient descent)
- scales better to larger data sets
- specify loss
- hyperparameter 'alpha' is like '1/C'
    - bigger alpha > more regularization

In [207]:
# Review: SGDClassfier for logreg vs SVM
from sklearn.linear_model import SGDClassifier
logreg = SGDClassifier(loss='log')
linsvm = SGDClassifier(loss='hinge')

In [208]:
from sklearn.linear_model import SGDClassifier

# We set random_state=0 for reproducibility 
linear_classifier = SGDClassifier(random_state=0)

# Instantiate the GridSearchCV object and run the search
# Search over the regularization strength, the hinge vs. log losses,
# and L1 vs. L2 regularization.
parameters = {'alpha':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 
             'loss':['hinge','log'], 'penalty':['l1','l2']}
searcher = GridSearchCV(linear_classifier, parameters, cv=10)
searcher.fit(X_train, y_train)

# Report the best parameters and the corresponding train and test scores
print("Best CV params", searcher.best_params_)
print("Best CV accuracy", searcher.best_score_)
print("Train accuracy of best grid search hypers:", 
      searcher.score(X_train,y_train))
print("Test accuracy of best grid search hypers:", 
      searcher.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

Best CV params {'alpha': 1e-05, 'loss': 'log', 'penalty': 'l1'}
Best CV accuracy 0.46408839779005523
Train accuracy of best grid search hypers: 0.5082872928176796
Test accuracy of best grid search hypers: 0.4098360655737705


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\sit

# 6. Decision Tree models
- CART - classification tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# adjust max_depth #########

# Instantiate the classifier: dt_clf_4
# :param max_depth of 4. This parameter specifies the maximum number 
#  of successive split points you can have before reaching a leaf node.
dt_clf_4 = DecisionTreeClassifier(max_depth=4)

# Fit the classifier to the training set
dt_clf_4.fit(X_train, y_train)

# Predict the labels of the test set: y_pred_4
y_pred_4 = dt_clf_4.predict(X_test)

# Compute the accuracy of the predictions: accuracy
accuracy = float(np.sum(y_pred_4==y_test))/y_test.shape[0]
print("accuracy:", accuracy)

# 7. XGBoost 
- good for 1000 training samples and <100 features
- ok when # of features < number of training samples
- issues if <100 training samples, or training sample size significantly smaller than # features

objective param: (default=reg:linear)
- multi:softmax, set num_class
- multi:softprob - result has predicted probability of each class

cross-validation: eval_metric - use error for classification
- logloss - negative log-likelihood
- merror - multiclass classification error rate
- mlogloss - multiclass logloss

Documentation
- https://xgboost.readthedocs.io/en/latest/python/python_api.html
Windows install:
- wheel file [here](https://www.lfd.uci.edu/~gohlke/pythonlibs/#xgboost)
- install directions [here](https://medium.com/@rakshithvasudev/how-i-installed-xgboost-after-a-lot-of-hassels-on-my-windows-machine-c53e972e801e)

In [360]:
# import packages
import xgboost as xgb


In [372]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [373]:
print("Train Accuracy: {}".format(xgb.score(X_train,y_train)))
print("Test Accuracy: {}".format(xgb.score(X_test, y_test)))

Train Accuracy: 1.0
Test Accuracy: 0.8979591836734694


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [368]:
X_train.shape

(193, 131)

### Overfit: training accuracy 1.0, test accuracy .898, overfit. Samples too small. Try a lot of subjects.

In [ ]:
# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy))

In [ ]:
# cross validation: for xgb cv, first explicitly convert data into a DMatrix

# Create the DMatrix: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:logistic", "max_depth":3}

# Perform cross-validation: cv_results
# "error" metrics will be converted to accuracy
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, nfold=3, 
                    num_boost_round=5, metrics="error", as_pandas=True, 
                    seed=123)

# Print cv_results
print(cv_results)

# Print the accuracy
print(((1-cv_results["test-error-mean"]).iloc[-1]))


In [ ]:
# Untuned Model Example
import pandas as pd
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X,y = housing_data[housing_data.columns.tolist()[:-1]],
      housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# most basic parameters for regression
untuned_params={"objective":"reg:linear"}

untuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix, 
                                 params=untuned_params,nfold=4,
                                 metrics="rmse",
                                 as_pandas=True, seeds=123)
    
print("Untuned rmse: %f" %((untuned_cv_results_rmse["test-rmse-mean"]) \
                          .tail(1)))
# code got cut off on the end of the line

# Untuned rmse: 34624.229980

In [ ]:
# Tuned Model Example
import pandas as pd
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X,y = housing_data[housing_data.columns.tolist()[:-1]],
      housing_data[housing_data.columns.tolist()[-1]]
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# tuned parameters - few xgb parameters that are important
tuned_params={"objective":"reg:linear", 'colsample_bytree':0.3,
             'learning_rate':0.1, 'max_depth':5}

# cross validation with 200 constructed trees
tuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix, 
                               params=tuned_params,nfold=4,
                               num_boost_round=200,
                               metrics="rmse",
                               as_pandas=True, seeds=123)
    
print("Tuned rmse: %f" %((tuned_cv_results_rmse["test-rmse-mean"]) \
                         .tail(1)))

In [ ]:
# Automated boosting round selection using early_stopping

# Create your housing DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree: params
params = {"objective":"reg:linear", "max_depth":4}

# Perform cross-validation with early stopping: cv_results
cv_results = xgb.cv(dtrain=housing_dmatrix,params=params,
                    early_stopping_rounds=10, num_boost_round=50, 
                    metrics="rmse",as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

## 7.1 XGB hyperparameters
Common Tree tunable parameters (most frequently used)

- learning rate: learning rate/eta
- gamma: min loss reduction to cretae new tree split
- lambda: L2 reg on leaf weights
- alpha: L1 reg on leaf weights
- max_depth: max depth per tree, must be positive integer
- subsample: % samples used per tree, value b/n 0 and 1
    - low value >> fraction of training data used per boosting round would be low >> possible underfitting
    - high value >> may lead to overfitting
- colsample_bytree: % features used per tree, value b/n 0 and 1
    - large value >> almost all features used
    - small value >> small subset of features
    - generally, smaller value can provide additional regularization
    - large value may overfit

Linear tunable parameters
- lambda: L2 reg on weights
- alpha: L1 reg on weights
- lambda_bias: L2 reg term on bias

Note: You can also tune the number of estimators used for both base model types (Tree and Linear)

In [ ]:
# Grid Search for hyperparameter turning, can use Random Search

housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
X, y = housing_data[housing_data.columns.tolist()[:-1]],
        housing_data[housing_data.columns.tolist()[-1]
housing_dmatrix = xgb.DMatrix(data=X,label=y)
                     
# 4 learning rate/eta values, 1 number of trees, 3 subsamples = 12 models
gbm_param_grid = {'learning_rate': [0.01,0.1,0.5,0.9],
                  'n_estimators': [200],
                  'subsample': [0.3, 0.5, 0.9]}
                     
gbm = xgb.XGBRegressor()
grid_mse = GridSearchCV(estimator=gbm,
                        param_grid=gbm_param_grid, 
                        scoring='neg_mean_squared_error', cv=4, verbose=1)
grid_mse.fit(X, y)
                     
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate': np.arange(0.05, 1, 0.05),
    'clf__max_depth': np.arange(3, 10, 1),
    'clf__n_estimators': np.arange(50, 200, 50)
}

# Perform RandomizedSearchCV
randomized_roc_auc = RandomizedSearchCV(estimator=pipeline, 
                                        param_distributions=gbm_param_grid,
                                        n_iter=2, 
                                        scoring="roc_auc", 
                                        verbose=1)

# Fit the estimator
randomized_roc_auc.fit(X, y)

# Compute metrics
print("Best score: ", randomized_roc_auc.best_score_)
print("Best estimator: ", randomized_roc_auc.best_estimator_)

## 7.2 Visualize trees

In [ ]:
# Create the DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:linear", "max_depth":2}

# Train the model: xg_reg
xg_reg = xgb.train(params=params, dtrain=housing_dmatrix, num_boost_round=10)

# Plot the first tree
xgb.plot_tree(xg_reg, num_trees=0)
plt.show()

# Plot the fifth tree
xgb.plot_tree(xg_reg, num_trees=4)
plt.show()

# Plot the last tree sideways
xgb.plot_tree(xg_reg, num_trees=4, rankdir="LR")
plt.show()

## 7.3 Visualize feature importance

In [ ]:
# Create the DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:linear", "max_depth":4}

# Train the model: xg_reg
xg_reg = xgb.train(params=params, dtrain=housing_dmatrix, num_boost_round=10)

# Plot the feature importances
xgb.plot_importance(xg_reg)
plt.show()

# 8. RandomForest

In [374]:
# Train model

# Import the model
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data (features and targets)
rf.fit(X_train, y_train);

In [375]:
# Make predictions on test set

# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.55 degrees.


In [377]:
# Determine performance metrics

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: -inf %.


In [ ]:
# Hyperparameter tuning to improve model



In [379]:
# Interpret model and report results

# Visualize a single decision tree
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

# Decrease layers of tree from 15 to 3
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

NameError: name 'feature_list' is not defined

# 9. Naive Bayes

In [349]:
# training a Naive Bayes classifier 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
gnb_predictions = gnb.predict(X_test) 
  
# accuracy on X_test 
accuracy = gnb.score(X_test, y_test) 
print(accuracy)
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, gnb_predictions) 

0.4262295081967213


# 10. Hierarchical Classification

In [31]:
# naming convention
# X samples (n x m array), aka data points or "singleton clusters"
# n number of samples
# m number of features
# Z cluster linkage array (contains the hierarchical clustering information)
# k number of clusters

In [ ]:
#from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
#import numpy as np
# some setting for this notebook to actually show the graphs inline
# you probably won't need this
#%matplotlib inline
#np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation

In [ ]:
# generate the linkage matrix
Z = linkage(X, 'ward')
# other common linkage methods: 'single', 'complete', 'average'

fromfrom  scipy.cluster.hierarchyscipy.c  import cophenet
from scipy.spatial.distance import pdist
c, coph_dists = cophenet(Z, pdist(X))
c



In [ ]:
idxs = [33, 68, 62]
plt.figure(figsize=(10, 8))
plt.scatter(X[:,0], X[:,1])  # plot all points
plt.scatter(X[idxs,0], X[idxs,1], c='r')  # plot interesting points in red again
plt.show()

idxs = [33, 68, 62]
plt.figure(figsize=(10, 8))
plt.scatter(X[:,0], X[:,1])
plt.scatter(X[idxs,0], X[idxs,1], c='r')
idxs = [15, 69, 41]
plt.scatter(X[idxs,0], X[idxs,1], c='y')
plt.show()



In [ ]:
# plot dendogram
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
# dendrogram truncation
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=12,  # show only the last p merged clusters
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show()

In [ ]:
# select distance cut-off to determine number of clusters
# set cut-off to 50
max_d = 50  # max_d as in max_distance


fancy_dendrogramfancy_de (
    Z,
    truncate_mode='lastp',
    p=12,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10,
    max_d=max_d,  # plot a horizontal cut-off line
)
plt.show()

In [ ]:
# Retrieve clusters

# knowing max_d
from scipy.cluster.hierarchy import fcluster
max_d = 50
clusters = fcluster(Z, max_d, criterion='distance')
clusters

# knowing k
k=2
fclusterfcluster(Z, k, criterion='maxclust')


In [ ]:
# Visualize clusters
plt.figure(figsize=(10, 8))
plt.scatter(X[:,0], X[:,1], c=clusters, cmap='prism')  # plot points with cluster dependent colors
plt.show()

# 